In [1]:
import pandas as pd
from tqdm import tqdm
# import gensim
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from joblib import Parallel, delayed
import sys
from scipy.stats import norm

In [2]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# # 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
# tf.config.set_visible_devices([], 'GPU')

from tqdm.notebook import tqdm
tf.random.set_seed(seed_value)
import multiprocessing


No GPU found


In [3]:
import matplotlib.pyplot as plt

import pickle

In [4]:
path = os.getcwd() + "//dataset_1"

In [5]:
def ann(hidden = 10, input_shape = 90): 
    input_layer = tf.keras.layers.Input(shape = (input_shape))
    hidden_layer = tf.keras.layers.Dense(hidden)
    hidden_output = hidden_layer(input_layer)
    pred_layer = tf.keras.layers.Dense(1)
    pred_output = pred_layer(hidden_output)
#     pred_output = tf.keras.layers.Activation("softmax")(pred_output)
    model = tf.keras.models.Model(input_layer, pred_output)
    return model

In [6]:
# X_train_word2vec = train_drugs

In [7]:
h1, h2 = 16,16

In [8]:


ann_15 = ann(h1)


# In[35]:


# ann_15.summary()


# In[36]:



In [9]:
ann_15.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 90)]              0         
                                                                 
 dense (Dense)               (None, 16)                1456      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,473
Trainable params: 1,473
Non-trainable params: 0
_________________________________________________________________


In [10]:
# X_train_word2vec = train_omics

In [11]:
ann_20 = ann(h2, 90)



# ann_20.summary()



In [12]:
total_weights_1 = ann_15.count_params()

In [13]:
total_weights_2 = ann_20.count_params()

In [14]:
total_weights = total_weights_1 + total_weights_2 + 1 

In [15]:
total_weights

2947

In [16]:
## batch size
batch_size = 128

In [17]:
# var_weights = 0.08
# var_targets = 0.08

In [18]:
from scipy.stats import multivariate_normal as mvn

In [19]:
from scipy.stats import invgamma, norm

In [20]:
from tqdm.notebook import tqdm

In [21]:
threshold = 5

In [22]:
import random

In [23]:
lr = 0.1

In [24]:
import numpy as np

In [25]:
weights_ann_1 = ann_15.get_weights()
weights_ann_2 = ann_20.get_weights()

In [26]:
def get_targets_with_weights(batch_data, batch_data1, initial_ensembles, log_sigma_points_1, size_ens): 

    n_hidden_1 = len(weights_ann_1[0].ravel())

    hidden_weights_1 = initial_ensembles[:,:n_hidden_1].reshape( size_ens, batch_data.shape[1], h1)

    
    hidden_output_1 = np.einsum('ij,kjl->kil', batch_data, hidden_weights_1)

    
    hidden_layer_bias_1 = initial_ensembles[:,n_hidden_1:(n_hidden_1 + h1)].reshape(size_ens, 1,  h1)



    hidden_output_1 = hidden_output_1 + hidden_layer_bias_1

    n_pred_weights_1 = len(weights_ann_1[2].ravel())

    output_weights_1 = initial_ensembles[:,(n_hidden_1 + h1):(n_hidden_1 + h1 + n_pred_weights_1) ].reshape(size_ens, h1, 1)

    output_1 = np.einsum('ijk,ikl->ijl', hidden_output_1, output_weights_1)


    output_layer_bias_1 = initial_ensembles[:,(n_hidden_1 + h1 + n_pred_weights_1):(n_hidden_1 + h1 + n_pred_weights_1 + 1)].reshape(size_ens, 1, 1)



    final_output_1 = output_1 + output_layer_bias_1

    n_hidden_2 = len(weights_ann_2[0].ravel())

    initial_ensembles_1 = initial_ensembles.copy()[:, total_weights_1:(total_weights_1+ total_weights_2)]
    
    # print(batch_data1.shape[1])
    
    
    hidden_weights_2 = initial_ensembles_1[:,:n_hidden_2].reshape(size_ens, batch_data1.shape[1], h2)



    hidden_output_2 = np.einsum('ij,kjl->kil', batch_data1, hidden_weights_2)

    hidden_layer_bias_2 = initial_ensembles[:,n_hidden_2:(n_hidden_2 + h2)].reshape(size_ens, 1,  h2)

    hidden_output_2 = hidden_output_2+ hidden_layer_bias_2

    n_pred_weights_2 = len(weights_ann_2[2].ravel())

    output_weights_2 = initial_ensembles_1[:,(n_hidden_2 + h2):(n_hidden_2 + h2 + n_pred_weights_2) ].reshape(size_ens, h2, 1)


    output_2 = np.einsum('ijk,ikl->ijl', hidden_output_2, output_weights_2)


    output_layer_bias_2 = initial_ensembles_1[:,(n_hidden_2 + h2 + n_pred_weights_2):(n_hidden_2 + h2 + n_pred_weights_2 + 1)].reshape(size_ens, 1, 1)


    final_output_2 = output_2 + output_layer_bias_2


    weights_1 = initial_ensembles[:, :total_weights_1]

    weights_2 = initial_ensembles[:, total_weights_1:(total_weights_1 + total_weights_2)]


    avg_weights = initial_ensembles[:, -1].reshape(-1,1)

    avg_weights_sig = expit(avg_weights)
    
    avg_weights_sig = avg_weights_sig.reshape(avg_weights_sig.shape[0], 1, avg_weights_sig.shape[1])
    
    complement_weights_sig = 1 - expit(avg_weights)
    
    complement_weights_sig = complement_weights_sig.reshape(complement_weights_sig.shape[0], 1, complement_weights_sig.shape[1])

    final_output_1 = final_output_1*complement_weights_sig
    
    final_output_2 = final_output_2*avg_weights_sig
    
    output_1_ravel = final_output_1.reshape(size_ens, final_output_1.shape[1]*final_output_1.shape[2])

    output_2_ravel = final_output_2.reshape(size_ens, final_output_2.shape[1]*final_output_2.shape[2])


    output_1_ravel = output_1_ravel

    output_2_ravel = output_2_ravel



    weights_1_add = np.zeros((size_ens, (total_weights_2 - total_weights_1)))



    weights_1 = np.hstack((weights_1, weights_1_add))
    


    stack_1 = np.hstack((output_1_ravel, weights_1, np.repeat(0, size_ens).reshape(-1,1), np.repeat(0, size_ens).reshape(-1,1)))



    
    stack_2 = np.hstack((output_2_ravel, weights_2, avg_weights, log_sigma_points_1))

    
    initial_aug_state = np.hstack((stack_1, stack_2)) 
    

    return initial_aug_state , output_1_ravel, output_2_ravel, log_sigma_points_1



In [27]:
import time

In [28]:
from scipy.stats import multivariate_normal as mvn

In [29]:
from scipy.stats import multivariate_normal as mvn

In [30]:
# train_fudged_probs

In [31]:
import matplotlib.pyplot as plt

In [32]:
from scipy.special import expit

In [33]:
# catch1[0][0]

In [34]:
# catch_train_logits_second[0]

In [35]:
from datetime import datetime

In [36]:
def expit(x):
    """Compute softmax values for each sets of scores in x."""
#     e_x = np.exp(x - np.max(x))
    return 1 / (1 + np.exp(-x))

In [37]:
from scipy.stats import gamma

In [38]:
use_cores = multiprocessing.cpu_count()-1

In [39]:
use_cores

47

In [40]:
from datetime import datetime

In [41]:
from sklearn.metrics import mean_squared_error

In [42]:
import random

In [43]:
from scipy import stats

In [44]:
from scipy.stats import pearsonr

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
from sklearn.model_selection import KFold

In [47]:
with open("..//..//Data//untouched_all_embs_five_fold.pickle", "rb") as f:
    catch_main = pickle.load(f)

In [48]:
# catch_main[0][0][0].shape

(800, 100)

In [52]:
# catch_main[0][1][1].shape

In [284]:
from sklearn.decomposition import PCA

In [285]:
# var_weights =1.0
# gamma_param = 100
# var_targetsw = 0.0005
# reduction = 15
# shape_needed = total_weights//reduction
# # size_ens = 196
# size_ens = int(shape_needed)
# batch_size = 1024
# threshold = 15

In [286]:
#     item = catch_main[0]
#     a1, a2 = item[0], item[1]
#     train_drugs_emb, train_gen_expr_emb, train_dna_methyl_emb, train_gen_mut_emb, train_y = a1
#     test_drugs_emb, test_gen_expr_emb, test_dna_methyl_emb, test_gen_mut_emb, test_y = a2
#     train_rmse = []
#     test_rmse = []
#     train_cov = []
#     test_cov = []
#     train_p = []
#     test_p = []
#     train_width = []
#     test_width = []
# #     from scipy.special import expit
#     patience_smaller = 0
#     patience_uns = 0
    
#     print("Size_Ens is " + str(size_ens), flush = True)
    
#     best_train_mae = 100000
    
#     best_train_width = 100000

        
#     X_train_logits_unshuffled = train_y.reshape(-1,1)
#     train_size = X_train_logits_unshuffled.shape[0]

#     X_test_logits = test_y.reshape(-1,1)
    
#     ## create training batch chunks
#     train_idx = list(range(0, len(X_train_logits_unshuffled)))
#     batch_chunks = [train_idx[i:i+batch_size] for i in range(0,len(X_train_logits_unshuffled),batch_size)]

#     ## generate some augmented variable for iteration 0
#     initial_aug_state_mean = np.repeat(0, total_weights)
#     initial_aug_state_mean = initial_aug_state_mean.reshape(-1,1)

#     initial_aug_state_cov = var_weights*np.identity((total_weights))
#     initial_ensembles = mvn(initial_aug_state_mean.reshape(initial_aug_state_mean.shape[0],), initial_aug_state_cov).rvs(size = size_ens)

#     exit_iter_no_thresh = 0
#     log_sigma_points_1 = (np.log(gamma(gamma_param, scale = 1/100).rvs(size_ens))).reshape(size_ens, 1)
    
    
#     train_lstm = train_drugs_emb

#     test_lstm = test_drugs_emb

#     pca = PCA(n_components=size_drugs)
#     train_lstm = pca.fit_transform(train_lstm)
#     # valid_lstm = pca.transform(valid_lstm)
#     test_lstm = pca.transform(test_lstm)
    
#     # print(train_lstm.shape)

#     train_doc2vec_gen_expr = train_gen_expr_emb
#     test_doc2vec_gen_expr = test_gen_expr_emb
    
#     pca = PCA(n_components=size_omics)
#     train_doc2vec_gen_expr = pca.fit_transform(train_doc2vec_gen_expr)
#     test_doc2vec_gen_expr = pca.transform(test_doc2vec_gen_expr)
    
#     train_doc2vec_gen_mut = train_gen_mut_emb
#     test_doc2vec_gen_mut = test_gen_mut_emb
    
#     pca = PCA(n_components=size_omics)
#     train_doc2vec_gen_mut = pca.fit_transform(train_doc2vec_gen_mut)
#     test_doc2vec_gen_mut = pca.transform(test_doc2vec_gen_mut)
    
#     train_doc2vec_dna_methyl = train_dna_methyl_emb
#     test_doc2vec_dna_methyl = test_dna_methyl_emb
    
#     pca = PCA(n_components=size_omics)
#     train_doc2vec_dna_methyl = pca.fit_transform(train_doc2vec_dna_methyl)
#     test_doc2vec_dna_methyl = pca.transform(test_doc2vec_dna_methyl)
    
#     train_drugs = train_lstm
    
#     test_drugs = test_lstm
    
#     train_omics = np.hstack((train_doc2vec_gen_expr, train_doc2vec_gen_mut, train_doc2vec_dna_methyl))

#     test_omics = np.hstack((test_doc2vec_gen_expr, test_doc2vec_gen_mut, test_doc2vec_dna_methyl))
    
    
#     train_lstm = train_drugs
#     test_lstm = test_drugs
    
#     train_doc2vec = train_omics
#     test_doc2vec = test_omics
    
#     best_coverage_train = 0
    
#     start = datetime.now()
    
#     for iter1 in range(0,1):
#         random_idx = random.sample(range(0,train_lstm.shape[0]), train_lstm.shape[0])
#         train_valid_lstm =train_lstm[random_idx, :]
#         train_valid_doc2vec = train_doc2vec[random_idx, :]
#         X_train_logits = X_train_logits_unshuffled[random_idx, :]

#         for batch_idx in batch_chunks:
            
#             batch_data = train_valid_lstm[batch_idx,:]
#             # print(batch_data.shape)
#             batch_data1 = train_valid_doc2vec[batch_idx,:]
#             # print(batch_data1.shape)
#             batch_targets = X_train_logits[batch_idx,:]
            
#             # print(batch_targets.shape)
            
#             # batch_targets = batch_targets.ravel().reshape(-1,1)

#             column_mod_2_shape = total_weights_2 + batch_data.shape[0]*1 + 1 + 1
        
#             H_t = np.hstack((np.identity(batch_targets.shape[0]), np.zeros((batch_targets.shape[0], column_mod_2_shape-batch_targets.shape[0]))))
            
            
            
            
#             current_aug_state, column_mod_1, column_mod_2, log_sigma_points_1 = get_targets_with_weights(batch_data1, batch_data, initial_ensembles, log_sigma_points_1, size_ens)
            
#             # print(current_aug_state.shape)
            
            
#             var_targets_vec = np.log(1 + np.exp(log_sigma_points_1))
            
#             var_targets_vec = var_targets_vec
            
#             # current_aug_state_var = np.cov(current_aug_state.T) + inflation_factor*np.identity(current_aug_state.shape[1])
            
#             current_aug_state_var = np.cov(current_aug_state.T) 
            
#             G_t = np.array([1 , 1]).reshape(-1,1)
            
#             scirpt_H_t = np.kron(G_t.T, H_t)
            
#             temp1 = current_aug_state_var@scirpt_H_t.T
            
#             temp2 = scirpt_H_t@current_aug_state_var@scirpt_H_t.T

#             R_t1 = var_targetsw*np.identity(current_aug_state.shape[1])
            
#             measurement_error1 = mvn(np.repeat(0,current_aug_state.shape[1]), R_t1).rvs(current_aug_state.shape[0])
            
#             # print("done")
            
#             R_t_batch = var_targets_vec.reshape(var_targets_vec.shape[0],1,1)*np.identity(batch_targets.shape[0])
#             measurement_error = mvn(np.repeat(0,batch_targets.shape[0]), R_t_batch.mean(0)).rvs(size_ens)
#             target_current = batch_targets.reshape(1,-1) + measurement_error
#             temp2 = np.dstack([temp2]*size_ens).T
#             K_t = temp1@np.linalg.inv(temp2 + R_t_batch)
#             current_aug_state = current_aug_state + ((K_t@np.expand_dims((target_current -(scirpt_H_t@current_aug_state.T).T),-1))[:,:,0])
#             current_aug_state = current_aug_state + measurement_error1

In [292]:
def rep_one_real_world(idx, size_ens, size_drugs = 90, size_omics = 30): 
    item = catch_main[idx]
    a1, a2 = item[0], item[1]
    train_drugs_emb, train_gen_expr_emb, train_dna_methyl_emb, train_gen_mut_emb, train_y = a1
    test_drugs_emb, test_gen_expr_emb, test_dna_methyl_emb, test_gen_mut_emb, test_y = a2
    train_rmse = []
    test_rmse = []
    train_cov = []
    test_cov = []
    train_p = []
    test_p = []
    train_width = []
    test_width = []
#     from scipy.special import expit
    patience_smaller = 0
    patience_uns = 0
    
    print("Size_Ens is " + str(size_ens), flush = True)
    
    best_train_mae = 100000
    
    best_train_width = 100000

        
    X_train_logits_unshuffled = train_y.reshape(-1,1)
    train_size = X_train_logits_unshuffled.shape[0]

    X_test_logits = test_y.reshape(-1,1)
    
    ## create training batch chunks
    train_idx = list(range(0, len(X_train_logits_unshuffled)))
    batch_chunks = [train_idx[i:i+batch_size] for i in range(0,len(X_train_logits_unshuffled),batch_size)]

    ## generate some augmented variable for iteration 0
    initial_aug_state_mean = np.repeat(0, total_weights)
    initial_aug_state_mean = initial_aug_state_mean.reshape(-1,1)

    initial_aug_state_cov = var_weights*np.identity((total_weights))
    initial_ensembles = mvn(initial_aug_state_mean.reshape(initial_aug_state_mean.shape[0],), initial_aug_state_cov).rvs(size = size_ens)

    exit_iter_no_thresh = 0
    log_sigma_points_1 = (np.log(gamma(gamma_param, scale = 1/100).rvs(size_ens))).reshape(size_ens, 1)
    
    
    train_lstm = train_drugs_emb

    test_lstm = test_drugs_emb

    pca = PCA(n_components=size_drugs)
    train_lstm = pca.fit_transform(train_lstm)
    # valid_lstm = pca.transform(valid_lstm)
    test_lstm = pca.transform(test_lstm)
    
    # print(train_lstm.shape)

    train_doc2vec_gen_expr = train_gen_expr_emb
    test_doc2vec_gen_expr = test_gen_expr_emb
    
    pca = PCA(n_components=size_omics)
    train_doc2vec_gen_expr = pca.fit_transform(train_doc2vec_gen_expr)
    test_doc2vec_gen_expr = pca.transform(test_doc2vec_gen_expr)
    
    train_doc2vec_gen_mut = train_gen_mut_emb
    test_doc2vec_gen_mut = test_gen_mut_emb
    
    pca = PCA(n_components=size_omics)
    train_doc2vec_gen_mut = pca.fit_transform(train_doc2vec_gen_mut)
    test_doc2vec_gen_mut = pca.transform(test_doc2vec_gen_mut)
    
    train_doc2vec_dna_methyl = train_dna_methyl_emb
    test_doc2vec_dna_methyl = test_dna_methyl_emb
    
    pca = PCA(n_components=size_omics)
    train_doc2vec_dna_methyl = pca.fit_transform(train_doc2vec_dna_methyl)
    test_doc2vec_dna_methyl = pca.transform(test_doc2vec_dna_methyl)
    
    train_drugs = train_lstm
    
    test_drugs = test_lstm
    
    train_omics = np.hstack((train_doc2vec_gen_expr, train_doc2vec_gen_mut, train_doc2vec_dna_methyl))

    test_omics = np.hstack((test_doc2vec_gen_expr, test_doc2vec_gen_mut, test_doc2vec_dna_methyl))
    
    
    train_lstm = train_drugs
    test_lstm = test_drugs
    
    train_doc2vec = train_omics
    test_doc2vec = test_omics
    
    best_coverage_train = 0
    
    start = datetime.now()
    
    for iter1 in range(0,500):
        random_idx = random.sample(range(0,train_lstm.shape[0]), train_lstm.shape[0])
        train_valid_lstm =train_lstm[random_idx, :]
        train_valid_doc2vec = train_doc2vec[random_idx, :]
        X_train_logits = X_train_logits_unshuffled[random_idx, :]
        
        
        
        for batch_idx in batch_chunks:
            start_inner = datetime.now()
            batch_data = train_valid_lstm[batch_idx,:]
            # print(batch_data.shape)
            batch_data1 = train_valid_doc2vec[batch_idx,:]
            # print(batch_data1.shape)
            batch_targets = X_train_logits[batch_idx,:]
            
            # print(batch_targets.shape)
            
            # batch_targets = batch_targets.ravel().reshape(-1,1)

            column_mod_2_shape = total_weights_2 + batch_data.shape[0]*1 + 1 + 1
        
            H_t = np.hstack((np.identity(batch_targets.shape[0]), np.zeros((batch_targets.shape[0], column_mod_2_shape-batch_targets.shape[0]))))
            
            
            
            
            current_aug_state, column_mod_1, column_mod_2, log_sigma_points_1 = get_targets_with_weights(batch_data1, batch_data, initial_ensembles, log_sigma_points_1, size_ens)
            
            # print(current_aug_state.shape)
            
            
            var_targets_vec = np.log(1 + np.exp(log_sigma_points_1))
            
            var_targets_vec = var_targets_vec
            
            # current_aug_state_var = np.cov(current_aug_state.T) + inflation_factor*np.identity(current_aug_state.shape[1])
            
            current_aug_state_var = np.cov(current_aug_state.T) 
            
            G_t = np.array([1 , 1]).reshape(-1,1)
            
            scirpt_H_t = np.kron(G_t.T, H_t)
            
            temp1 = current_aug_state_var@scirpt_H_t.T
            
            temp2 = scirpt_H_t@current_aug_state_var@scirpt_H_t.T

            R_t1 = var_targetsw*np.identity(current_aug_state.shape[1])
            
            measurement_error1 = mvn(np.repeat(0,current_aug_state.shape[1]), R_t1).rvs(current_aug_state.shape[0])
            
            R_t_batch = var_targets_vec.reshape(var_targets_vec.shape[0],1,1)*np.identity(batch_targets.shape[0])
            measurement_error = mvn(np.repeat(0,batch_targets.shape[0]), R_t_batch.mean(0)).rvs(size_ens)
            target_current = batch_targets.reshape(1,-1) + measurement_error
            temp2 = np.dstack([temp2]*size_ens).T
            K_t = temp1@np.linalg.inv(temp2 + R_t_batch)
            current_aug_state = current_aug_state + ((K_t@np.expand_dims((target_current -(scirpt_H_t@current_aug_state.T).T),-1))[:,:,0])
            current_aug_state = current_aug_state + measurement_error1

            weights_ann_1 = current_aug_state[:,batch_targets.shape[0]:(batch_targets.shape[0] + total_weights_1)]      

            weights_ann_2 = current_aug_state[:,-(total_weights_2+1):-2]    

            initial_ensembles = np.hstack((weights_ann_1, weights_ann_2, current_aug_state[:,-2].reshape(-1,1)))
            
            log_sigma_points_1 = current_aug_state[:,-1].reshape(-1,1)
               
            avg_betas = expit(current_aug_state[:,-2])
        
            complement = 1-avg_betas


            current_aug_state1, column_mod_11, column_mod_21, log_sigma_points_1 = get_targets_with_weights(train_doc2vec, train_lstm, initial_ensembles, log_sigma_points_1, size_ens)
            
            initial_targets = column_mod_11 + column_mod_21
            
            
            initial_targets = initial_targets.reshape(size_ens, train_lstm.shape[0],1)
            
            initial_targets_train = initial_targets
            
            initial_targets_train_mean = initial_targets_train.mean(0)
            
            li = np.percentile(initial_targets_train, axis = 0, q = (2.5, 97.5))[0,:,:]
            
            ui = np.percentile(initial_targets_train, axis = 0, q = (2.5, 97.5))[1,:,:]
            
            width = ui - li
            
            avg_width_train = np.mean(width)
            
            # interim = np.vstack((catch_train_probs[idx] , catch_valid_probs[idx] ))
        
            interim = (X_train_logits_unshuffled)
            
            ind = (interim >= li) & (interim <= ui)
            
            coverage_train= np.mean(ind.ravel())  
            
            train_mae = np.sqrt(mean_squared_error(interim, initial_targets_train_mean))
            
            # plt.scatter(interim, initial_targets_train_mean)
            # plt.show()
            
            stats_pearson_train = stats.pearsonr(interim.reshape(interim.shape[0],),
                                           initial_targets_train_mean.reshape(initial_targets_train_mean.shape[0],))

            # train_mae = np.sqrt(np.mean(np.abs(interim.ravel() - initial_targets_train_mean.ravel())**2))
        
            current_aug_state1, column_mod_11, column_mod_21, log_sigma_points_1 = get_targets_with_weights(test_doc2vec, test_lstm, initial_ensembles, log_sigma_points_1, size_ens)
            
            initial_targets = column_mod_11 + column_mod_21
            
            initial_targets = initial_targets.reshape(size_ens, test_lstm.shape[0],1)
            
            initial_targets_test = initial_targets
            
            # initial_targets_softmax = expit(initial_targets)    
            
            initial_targets_test_mean = initial_targets_test.mean(0)
            
            li = np.percentile(initial_targets_test, axis = 0, q = (2.5, 97.5))[0,:,:]
            
            ui = np.percentile(initial_targets_test, axis = 0, q = (2.5, 97.5))[1,:,:]
            
            width = ui - li
            
            avg_width = np.mean(width)
            
            catch_test_probs = (X_test_logits)
            
            ind_test = (catch_test_probs >= li) & (catch_test_probs <= ui)
                        
            coverage_test = np.mean(ind_test.ravel())    

            
            test_mae = np.sqrt(mean_squared_error(catch_test_probs, initial_targets_test_mean))

            stats_pearson = stats.pearsonr(catch_test_probs.reshape(catch_test_probs.shape[0],),
                                           initial_targets_test_mean.reshape(initial_targets_test_mean.shape[0],))

            
            start_inner_e = datetime.now()
            total = (start_inner_e- start_inner)
            time_taken = total.seconds/60.0
            # print(time_taken)
            
            train_rmse.append(train_mae)
            test_rmse.append(test_mae)
            train_cov.append(coverage_train)
            test_cov.append(coverage_test)
            train_p.append(stats_pearson_train)
            test_p.append(stats_pearson)
            train_width.append(avg_width_train)
            test_width.append(avg_width)
            
            if  train_mae < best_train_mae: 
                patience = 0
                best_train_mae = train_mae
                # best_valid_mae = valid_mae
                best_test_mae = test_mae
                exit_iter_no_thresh = iter1
                best_test_preds = initial_targets_test
                patience_smaller = 0
                best_train_coverage = coverage_train
                # best_valid_coverage = coverage_valid
                best_test_coverage = coverage_test
                best_train_width = avg_width_train
                # best_valid_width = avg_width_valid
                best_test_width = avg_width
                best_drugs_weight = np.mean(complement)
                best_r_test = stats_pearson[0]
                best_r_train = stats_pearson_train[0]
                # best_r_valid = stats_pearson_valid[0]
                best_initial_ensembles = initial_ensembles
                best_log_sigma_points_1 = log_sigma_points_1
                best_initial_targets_test_mean = initial_targets_test_mean
                best_li = li
                best_ui = ui
            else:
                patience += 1
                
#             print("epoch "+ str(iter1))
#             print("patience "+ str(patience))
#             print("train mae is " + str(train_mae))
#             # print("valid mae is " + str(valid_mae))
#             print("test mae is " + str(test_mae))
        
#             print("train coverage is "+ str(coverage_train))
#             # print("valid coverage is "+ str(coverage_valid))
#             print("test coverage is "+ str(coverage_test))
            
#             print("train width is " + str(avg_width_train))
#             # print("valid width is " + str(avg_width_valid))  
#             print("test width is " + str(avg_width))
            
#             print("drugs weight is " + str(np.mean(complement)))
#             print("pearson r train is " + str(stats_pearson_train[0]))
#             # print("pearson r valid is " + str(stats_pearson_valid[0]))
#             print("pearson r test is " + str(stats_pearson[0]))
            
            
                
            if patience >= threshold: 
                break
                
        if patience >= threshold: 
                break
            
    
    # print(best_valid_mae, best_valid_coverage, best_valid_width,best_r_valid, flush = True)
    print(best_test_mae, best_test_coverage, best_test_width,best_r_test, flush = True)
    end = datetime.now()
    total = (end- start)
    time_taken = total.seconds/60.0
    return size_ens, best_train_mae, best_test_mae, best_train_coverage, best_test_coverage,  \
    best_train_width , best_test_width,  best_drugs_weight, best_r_train, best_r_test, exit_iter_no_thresh, time_taken, best_initial_ensembles, best_log_sigma_points_1, [X_test_logits.tolist(), best_initial_targets_test_mean.tolist()], \
    [best_li.tolist(), best_ui.tolist()], train_rmse, test_rmse, train_cov, test_cov, train_p, test_p, train_width, test_width 
        
    # print("something went awry", flush = True)

In [288]:
size_drugs = 90
size_omics = 30

In [289]:
var_weights =1
gamma_param = 500
var_targetsw = 0.0002
reduction = 15
shape_needed = total_weights//reduction
# size_ens = 196
size_ens = int(shape_needed)
batch_size = 512
threshold = 15

In [290]:
size_ens

196

In [291]:
size_ens,best_train_mae, best_test_mae, best_train_coverage, best_test_coverage,  \
    best_train_width , best_test_width,  best_drugs_weight, best_r_train, best_r_test, exit_iter_no_thresh, time_taken, best_initial_ensembles, best_log_sigma_points_1, _, _, train_rmse, test_rmse, train_cov, test_cov, train_p, test_p, train_width, test_width = rep_one_real_world(0, size_ens)

Size_Ens is 196
epoch 0
patience 0
train mae is 35.61769001241577
test mae is 35.87263841526475
train coverage is 1.0
test coverage is 1.0
train width is 483.98282303279103
test width is 486.4397319704673
drugs weight is 0.41120155423422017
pearson r train is 0.08883535297484439
pearson r test is 0.10181993817916228
epoch 0
patience 0
train mae is 8.103432428816665
test mae is 8.043834661628377
train coverage is 1.0
test coverage is 1.0
train width is 207.90182655914404
test width is 208.28822509912308
drugs weight is 0.31834696464499473
pearson r train is 0.11826334950280043
pearson r test is 0.13833084310548405
epoch 1
patience 1
train mae is 9.08385607641182
test mae is 9.110488976169595
train coverage is 1.0
test coverage is 1.0
train width is 98.43521494920664
test width is 98.95695746372506
drugs weight is 0.2724325443703506
pearson r train is 0.1484039331431578
pearson r test is 0.14749786491960246
epoch 1
patience 0
train mae is 3.521360849396901
test mae is 3.5701843458158335


In [ ]:
# test_actual_preds[0].shape

In [ ]:
# len(test_actual_preds)

In [ ]:
# test_actual_preds, best_li_ui

In [293]:
from joblib import Parallel, delayed

In [ ]:
# test_actual_preds[:5]

In [ ]:
# best_li_ui[0]

In [295]:
catch = Parallel(n_jobs = 5, backend = "loky", verbose = 8)(delayed(rep_one_real_world)(idx, size_ens) for idx in range(0,5))

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


Size_Ens is 196
Size_Ens is 196
Size_Ens is 196
Size_Ens is 196
Size_Ens is 196
1.4675160276643655 0.97 6.90330354100128 0.6696029225043486
1.7448731988803483 0.93 6.560426228026753 0.5057288489980584


[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed: 14.4min remaining: 21.6min


1.8208862717475272 0.965 7.811751600066373 0.3899747192504727


[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed: 14.6min remaining:  9.7min


1.4559401625255952 0.985 7.023859174404155 0.5629642073828152
1.4593503178847835 0.995 10.21703474216474 0.6682756079905818


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 26.4min remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed: 26.4min finished


In [296]:
with open("Preds_with_PCA_proper_CV_no_valid_all_features_untouched_five_on_1000_cv" + ".pkl", 'wb') as f:
    pickle.dump(catch, f)

In [297]:
with open("Preds_with_PCA_proper_CV_no_valid_all_features_untouched_five_on_1000_cv" + ".pkl", 'rb') as f:
    catch = pickle.load(f)

In [298]:
col_names = ['size_ens','best_train_mae', 'best_test_mae', 'best_train_coverage', 'best_test_coverage',  \
    'best_train_width' , 'best_test_width',  'best_drugs_weight', 'best_r_train', 'best_r_test']

In [299]:
catch_df = pd.DataFrame(catch).iloc[:,:len(col_names)]

In [300]:
catch_df.columns = col_names

In [ ]:
# catch_df.columns = ['best_train_mae', 'best_test_mae', 'best_train_coverage', 'best_test_coverage',  \
#     'best_train_width' , 'best_test_width',  'best_drugs_weight', 'best_r_train', 'best_r_test']

In [301]:
catch_df.mean()

size_ens               196.000000
best_train_mae           1.563428
best_test_mae            1.589713
best_train_coverage      0.970750
best_test_coverage       0.969000
best_train_width         7.700806
best_test_width          7.703275
best_drugs_weight        0.412681
best_r_train             0.584428
best_r_test              0.559309
dtype: float64

In [ ]:
catch_df = []
for i in range(0, len(catch)): 
    item = catch[i]
    size_ens = item[0]
    train_rmse = item[-7]
    test_rmse = item[-8]
    df = pd.DataFrame({"Train_RMSE": train_rmse, 
                      "Test_RMSE": test_rmse})
    df["size_ens"] = size_ens
    df["update_iteration"] = np.array(range(0, df.shape[0])) + 1
    catch_df.append(df)

In [ ]:
catch_df = pd.concat(catch_df)

In [ ]:
catch_df.head()

In [ ]:
# catch_df = catch_df[catch_df["size_ens"] != 25]

In [ ]:
catch_df.dtypes

In [ ]:
sizes = [ 50, 100, 200, 300, 400]

In [ ]:
catch_df = catch_df[catch_df["size_ens"].isin(sizes)]

In [ ]:
# logical_ids = [True if (float(size_ens)%100) == 0 else False for size_ens in catch_df['size_ens'].values ]

In [ ]:
catch_df["size_ens"] = catch_df["size_ens"].astype(str)

In [ ]:
catch_df.shape

In [ ]:
# !pip install seaborn

In [ ]:
import seaborn as sns

In [ ]:
fig = plt.figure(figsize = (6,4))
sns.lineplot(data=catch_df, x="update_iteration", y="Train_RMSE", hue="size_ens", linewidth = 3)
plt.xlabel("Update Iteration", fontsize = 15, fontweight = "bold")
plt.ylabel("Train RMSE", fontsize = 15, fontweight = "bold")
plt.xticks(fontsize = 10, fontweight = "bold")
plt.yticks(fontsize = 10, fontweight = "bold")
leg = plt.legend(title = "Ensemble Size",fontsize=10, prop =  {'weight':'bold'}, labelcolor = "black", 
                title_fontproperties=  {'weight':'bold'})
for legobj in leg.legend_handles:
    legobj.set_linewidth(3.0)
fig.savefig('Plots/Different_Ensemble_Size.pdf', bbox_inches='tight', format = "pdf")
fig.savefig('Plots/Different_Ensemble_Size.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize = (6,3))
sns.lineplot(data=catch_df, x="update_iteration", y="Test_RMSE", hue="size_ens", linewidth = 4)
plt.xlabel("Update Iteration", fontsize = 15, fontweight = "bold")
plt.ylabel("Test RMSE", fontsize = 15, fontweight = "bold")
plt.xticks(fontsize = 10, fontweight = "bold")
plt.yticks(fontsize = 10, fontweight = "bold")
leg = plt.legend(title = "Ensemble Size",fontsize=10, prop =  {'weight':'bold'}, labelcolor = "black", 
                title_fontproperties=  {'weight':'bold'})
plt.show()

In [ ]:
# catch_df['best_drugs_weight'] = 1 - catch_df['best_drugs_weight']

In [ ]:
# catch_df.mean()

In [ ]:
# pca = PCA(n_components = test_omics.shape[1])
# pca.fit(train_drugs)

In [ ]:
# test_drugs = pca.transform(test_drugs)

In [ ]:
# test_drugs.shape

In [ ]:
# preds_test_catch = []
# test_catch = []
# li_catch = []
# ui_catch = []
# for i in range(0,6):
#     preds = catch[i][-2][1]
#     actual = catch[i][-2][0]
#     preds_test_catch.append(preds)
#     test_catch.append(actual)
# #     best_log_sigma_points_1 = catch[i][-3]
# #     current_aug_state1, column_mod_11, column_mod_21, log_sigma_points_1 = get_targets_with_weights(test_omics, test_drugs, params, best_log_sigma_points_1)
            
# #     initial_targets_test = column_mod_11 + column_mod_21
            
# #     initial_targets_test = initial_targets_test.reshape(size_ens, test_omics.shape[0],1)
            
# #     initial_targets_test_mean = initial_targets_test.mean(0)
    
#     # preds_test_catch.append(preds)
    
#     li = catch[i][-1][0]
#     li_catch.append(li)
            
#     ui = catch[i][-1][1]
#     ui_catch.append(ui)
            
# #     width = ui - li
            
# #     avg_width = np.mean(width)

In [ ]:
# preds_test_catch = [inner for item in preds_test_catch for inner in item]

In [ ]:
# test_catch = [inner for item in test_catch for inner in item]

In [ ]:
# li_catch =  [inner for item in li_catch for inner in item]

In [ ]:
# ui_catch =  [inner for item in ui_catch for inner in item]

In [ ]:
# avg_width

In [ ]:
# y_test_preds = np.array(preds_test_catch).mean(0)

In [ ]:
# y_test_preds.shape

In [ ]:
# ui

In [ ]:
# current_aug_state1, column_mod_11, column_mod_21, log_sigma_points_1 = get_targets_with_weights(test_omics, test_drugs, best_initial_ensembles, best_log_sigma_points_1)
            
# initial_targets = column_mod_11 + column_mod_21
            
# initial_targets = initial_targets.reshape(size_ens, test_omics.shape[0],1)
            
# initial_targets_test = initial_targets

In [ ]:
# # initial_targets_softmax = expit(initial_targets)    
            
# initial_targets_test_mean = initial_targets_test.mean(0)
            
# li = np.percentile(initial_targets_test, axis = 0, q = (2.5, 97.5))[0,:,:]
            
# ui = np.percentile(initial_targets_test, axis = 0, q = (2.5, 97.5))[1,:,:]
            
# width = ui - li
            
# avg_width = np.mean(width)
            
# catch_test_probs = (y_test)
            
# ind_test = (catch_test_probs >= li) & (catch_test_probs <= ui)
                        
# coverage_test = np.mean(ind_test.ravel())    

            
# test_mae = np.sqrt(mean_squared_error(catch_test_probs, initial_targets_test_mean))

# stats_pearson = stats.pearsonr(catch_test_probs.reshape(catch_test_probs.shape[0],),
#                                            initial_targets_test_mean.reshape(initial_targets_test_mean.shape[0]))

In [ ]:
# stats_pearson

In [ ]:
# string = "Real_World_EnKF_DeepCDR_" + "Neurons_" + str(h1)  + "_Gamma_" +  str(gamma_param) +  "_var_weights_" + str(var_weights) + "_num_ens_" + str(size_ens)

In [ ]:
# string = "First_Three_Real_World_EnKF_DeepCDR_" + "Neurons_" + str(h1)  + "_Gamma_" +  str(gamma_param) +  "_var_weights_" + str(var_weights) + "_num_ens_" + str(size_ens)

In [ ]:
# string

In [ ]:
# catch_coverages = [best_train_mae, best_valid_mae, best_test_mae, best_train_coverage, best_test_coverage, best_valid_coverage, best_test_coverage, \
#     best_train_width, best_valid_width , best_test_width, best_drugs_weight, best_r_train, best_r_valid, best_r, exit_iter_no_thresh, time_taken, best_initial_ensembles, best_log_sigma_points_1]

In [ ]:
# best_train_mae, best_valid_mae, best_test_mae, best_train_coverage, best_test_coverage, best_valid_coverage, best_test_coverage, \
#     best_train_width, best_valid_width , best_test_width, (1-best_drugs_weight), best_r_train, best_r_valid, best_r, exit_iter_no_thresh, time_taken, best_initial_ensembles, best_log_sigma_points_1

In [ ]:
# with open("Preds_Mutation_Methylation" + ".pkl", 'wb') as f:
#     pickle.dump(catch_coverages, f)

In [ ]:
# fig  = plt.figure()
# plt.scatter(y_test, initial_targets_test_mean, alpha = 0.5)
# plt.axline((0,0), slope = 1, c = "black", linewidth = 3)
# plt.xlabel("Observed Log IC50", fontweight = "bold", fontsize = 15)
# plt.ylabel("MEnKF-ANN Predicted Log IC50", fontweight = "bold", fontsize = 15)
# plt.xticks(fontsize = 15, fontweight = "bold")
# plt.yticks(fontsize = 15, fontweight = "bold")
# fig.savefig('Plots/Avg_Smile_Weight_Tajectory.pdf', bbox_inches='tight', format = "pdf")
# fig.savefig('Plots/Avg_Smile_Weight_Tajectory.png', bbox_inches='tight')
# plt.show()

In [ ]:
# catch_coverages

In [ ]:
# catch_coverages

In [ ]:
# check = pd.DataFrame(catch_coverages[:10])

In [ ]:
# check

In [ ]:
# check

In [ ]:
# check.columns = ["train_mae", "test_mae", "avg_ci_width_train", "avg_ci_width_test", "avg_lstm_weight", "exit_iter", "exit_status"]

In [ ]:
# check = check.T


In [ ]:
# check

In [ ]:
# check.columns = ["train_mae", "test_mae", "train_coverage", "train_width", "test_coverage", "test_width", "exit_iter", "time_taken", "avg_width_alginate"]

In [ ]:
# check.columns = ["train_mae", "test_mae", "train_coverage", "train_width", "test_coverage", "test_width", "drug_weight" , "pearson_correlation" ,"exit_iter", "time_taken"]

In [ ]:
# true_preds = [inner for item in true_preds for inner in item]

In [ ]:
# enkf_preds = [inner for item in enkf_preds for inner in item]

In [ ]:
# np.mean(np.abs(np.array(true_preds)-np.array(enkf_preds)))

In [ ]:
# plt.scatter(enkf_preds,true_preds, alpha=0.5)
# plt.xlabel("EnKF Predictions", fontsize = 15, fontweight = "bold")
# plt.ylabel("LSTM Predictions", fontsize = 15, fontweight = "bold")
# plt.xticks( fontsize = 10, fontweight = "bold")
# plt.yticks( fontsize = 10, fontweight = "bold")
# plt.axline((0, 0), slope=1)
# plt.show()

In [ ]:
# cur_idx = 0

In [ ]:
# catch_preds = []
# for i in range(0, catch_coverages[cur_idx][-1].shape[1]):
#     enkf_preds = catch_coverages[cur_idx][-1][:,i,:]
#     enkf_preds_df = pd.DataFrame(enkf_preds)
#     enkf_preds_df["Test_Sample_ID"] = i 
#     enkf_preds_df.columns = ["EnKF_Preds", "Test_Sample_ID"]
#     catch_preds.append(enkf_preds_df)

In [ ]:
# catch_preds_df = pd.concat(catch_preds)

In [ ]:
# import seaborn as sns

In [ ]:
# filtered_df = expit(catch1[cur_idx][2])

In [ ]:
# # plt.figure(figsize = (10,10))
# fig = sns.boxplot(catch_preds_df, x="Test_Sample_ID", y="EnKF_Preds", showfliers=False)
# plt.scatter(range(0, filtered_df.shape[0]), filtered_df, c = "black", s = 50)
# plt.xlabel("Test Sample ID", fontsize = 15, fontweight = "bold")
# plt.ylabel("EnKF Predictions", fontsize = 15, fontweight = "bold")
# plt.xticks(fontsize = 10, fontweight = "bold")
# plt.yticks(fontsize = 10, fontweight = "bold")
# plt.show()

In [ ]:
# ids = [i for i in range(0, filtered_df.shape[0]) if filtered_df[i] < 0.5]
# ids_more = [i for i in range(0, filtered_df.shape[0]) if filtered_df[i] > 0.5]

In [ ]:
# catch_preds_df.groupby("Test_Sample_ID").quantile(q = [0.025, 0.975])

In [ ]:
# catch_preds_df_filtered = catch_preds_df[catch_preds_df["Test_Sample_ID"].isin(ids_more)]

In [ ]:
# filtered_true = filtered_df[ids_more,:]

In [ ]:
# # plt.figure(figsize = (10,10))
# fig = sns.boxplot(catch_preds_df_filtered, x="Test_Sample_ID", y="EnKF_Preds", showfliers=False)
# plt.scatter(range(0, filtered_true.shape[0]), filtered_true, c = "black", s = 50)
# plt.xlabel("Test Sample ID", fontsize = 15, fontweight = "bold")
# plt.ylabel("EnKF Predictions", fontsize = 15, fontweight = "bold")
# plt.xticks(fontsize = 10, fontweight = "bold")
# plt.yticks(fontsize = 10, fontweight = "bold")
# plt.show()

In [ ]:
# catch_preds_df_filtered = catch_preds_df[catch_preds_df["Test_Sample_ID"].isin(ids)]
# filtered_true = filtered_df[ids,:]
# # plt.figure(figsize = (10,10))
# fig = sns.boxplot(catch_preds_df_filtered, x="Test_Sample_ID", y="EnKF_Preds", showfliers=False)
# plt.scatter(range(0, filtered_true.shape[0]), filtered_true, c = "black")
# plt.xlabel("Test Sample ID", fontsize = 15, fontweight = "bold")
# plt.ylabel("EnKF Predictions", fontsize = 15, fontweight = "bold")
# plt.xticks(fontsize = 10, fontweight = "bold")
# plt.yticks(fontsize = 10, fontweight = "bold")
# plt.show()
# # plt.ylim((0.9,1.0))